In [23]:
%pip install pandas
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import ast
import re
import joblib
import sqlite3

In [25]:
df = pd.read_csv('product_info.csv')

In [ ]:
# Cleaning original data
skin_df = df[df['primary_category'] == 'Skincare']
dropc_df = skin_df.drop(['variation_type', 'variation_value', 'variation_desc', 'value_price_usd', 
                          'sale_price_usd', 'limited_edition', 'new', 'out_of_stock', 'primary_category', 'tertiary_category', 
                          'child_count', 'child_max_price', 'child_min_price'], axis = 1)
new_df = dropc_df.dropna(subset = ['ingredients'])
new_df['new_size'] = new_df['size'].astype(str).apply(lambda x: x.split("/")[0])
new_df = new_df.drop(columns=['size']).rename(columns={'new_size': 'size'})

#new_df.to_csv('ingredients_df.csv', index = False)

/var/folders/mp/6v7lzsxx4jj_46cnmjxx9fv40000gn/T/ipykernel_6721/64277145.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['new_size'] = new_df['size'].astype(str).apply(lambda x: x.split("/")[0])


In [10]:
# Read in dataset with dummy variable for ingredients
product_df = pd.read_csv('skincare_with_skin_type_flags.csv')

In [ ]:
# fill missing values and clean data
product_df['size'] = product_df['size'].fillna("Size Unknown")
product_df['highlights'] = product_df['highlights'].fillna("[]")
product_df = product_df.rename(columns={'secondary_category': 'product_category'})
product_df = product_df[product_df['product_category'] != 'Shop by Concern']
product_df['price_usd'] = product_df['price_usd'].astype(int)

product_df['is_alcohol_free'] = product_df['contains_alcohol'].apply(lambda x: 1 if x == 0 else 0)
product_df['is_vegan'] = product_df['highlights'].str.contains('vegan', case=False, na=False).astype(int)
product_df['is_hypoallergenic'] = product_df['highlights'].str.contains('hypoallergenic', case=False, na=False).astype(int)

# convert to numeric types
product_df['rating'] = pd.to_numeric(product_df['rating'], errors='coerce').fillna(0.0).astype(float)
product_df['rating'] = product_df['rating'].round(2)
product_df['reviews'] = pd.to_numeric(product_df['reviews'], errors='coerce').fillna(0).astype(int)

# get dummy variable for primary_category column
one_hot = pd.get_dummies(product_df['product_category'], prefix='category').astype(int)

# combine with original dataset
product_df = pd.concat([product_df, one_hot], axis=1)

In [ ]:
def price_category(price):
    if price <= 15:
        return "0-15"
    elif price <= 30:
        return "16-30"
    elif price <= 45:
        return "31-45"
    elif price <= 60:
        return "46-60"
    else:
        return "61+"
    
product_df['price_category'] = product_df['price_usd'].apply(price_category)

In [ ]:
# clean and flatten ingredients
def clean_ingredients(text):
    try:
        ingredients = ast.literal_eval(text)
        flat = ', '.join(ingredients).lower()
        flat = re.sub(r'\([^)]*\)', '', flat)         # remove stuff in parentheses
        flat = re.sub(r'[^a-z\s]', ' ', flat)         # remove non-letter characters
        flat = re.sub(r'\s+', ' ', flat).strip()      # collapse extra whitespace
        return flat
    except:
        return ''
    

# clean and flatten highlights
def clean_highlights(text):
    try:
        highlights = ast.literal_eval(text)  # convert string to list
        flat = ', '.join(highlights).lower()  # lowercase and flatten
        flat = re.sub(r'[^a-z\s]', ' ', flat)  # remove non-letter characters
        flat = re.sub(r'\s+', ' ', flat).strip()  # collapse extra whitespace
        return flat
    except:
        return ''

# apply cleaning functions
product_df['ingredients_cleaned'] = product_df['ingredients'].apply(clean_ingredients)
product_df['highlights_cleaned'] = product_df['highlights'].apply(clean_highlights)

In [16]:
# TF-IDF on ingredients
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(product_df['ingredients'])

# TF-IDF on highlights
tfidf_highlights = TfidfVectorizer(stop_words='english')
highlights_matrix = tfidf_highlights.fit_transform(product_df['highlights_cleaned'])

In [48]:
product_df.head()

,product_id,product_name,brand_id,brand_name,loves_count,rating,reviews,ingredients,price_usd,online_only,...,category_Moisturizers,category_Self Tanners,category_Sunscreen,category_Treatments,category_Value & Gift Sets,category_Wellness,ingredients_cleaned,highlights_cleaned,combined_text,price_category
0,P439055,GENIUS Sleeping Collagen Moisturizer,6018,Algenist,33910,4.54,1321,"['Collagen (Vegan)*, Water (Aqua, Eau), Ethylh...",98,0,...,1,0,0,0,0,0,collagen water ethylhexyl palmitate oryza sati...,vegan good for loss of firmness collagen hypoa...,collagen water ethylhexyl palmitate oryza sati...,61+
1,P421277,GENIUS Liquid Collagen Serum,6018,Algenist,67870,4.03,1159,"['Collagen (Vegan)*, Water (Aqua, Eau), Propan...",115,0,...,0,0,0,1,0,0,collagen water propanediol isononyl isononanoa...,vegan good for loss of firmness collagen hypoa...,collagen water propanediol isononyl isononanoa...,61+
2,P467602,Triple Algae Eye Renewal Balm Eye Cream,6018,Algenist,17890,4.53,1142,"['Aqua (Water/Eau), Stearic Acid, Isopropyl Is...",68,0,...,0,0,0,0,0,0,aqua stearic acid isopropyl isostearate dimeth...,,aqua stearic acid isopropyl isostearate dimeth...,61+
3,P432045,GENIUS Liquid Collagen Lip Treatment,6018,Algenist,44448,3.87,649,"['Collagen (Vegan)*, Water (Aqua, Eau), Glycer...",29,0,...,0,0,0,0,0,0,collagen water glycerin isononyl isononanoate ...,vegan good for loss of firmness plumping colla...,collagen water glycerin isononyl isononanoate ...,16-30
4,P311143,SUBLIME DEFENSE Ultra Lightweight UV Defense F...,6018,Algenist,27278,4.41,508,"['Octinoxate 7.5%, Titanium Dioxide 2%, Zinc O...",28,0,...,0,0,1,0,0,0,octinoxate titanium dioxide zinc oxide cyclope...,vegan hypoallergenic uv protection spf best fo...,octinoxate titanium dioxide zinc oxide cyclope...,16-30


In [16]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2285 entries, 0 to 2285
Data columns (total 44 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   product_id                       2285 non-null   object 
 1   product_name                     2285 non-null   object 
 2   brand_id                         2285 non-null   int64  
 3   brand_name                       2285 non-null   object 
 4   loves_count                      2285 non-null   int64  
 5   rating                           2285 non-null   float64
 6   reviews                          2285 non-null   int64  
 7   ingredients                      2285 non-null   object 
 8   price_usd                        2285 non-null   float64
 9   online_only                      2285 non-null   int64  
 10  sephora_exclusive                2285 non-null   int64  
 11  highlights                       2285 non-null   object 
 12  product_category         

In [61]:
product_df.to_csv('complete_skincare_dataset.csv', index=False)

In [ ]:
db_df = pd.read_csv("complete_skincare_dataset.csv")
conn = sqlite3.connect("skincare_products.db")
db_df.to_sql("products", conn, if_exists="replace", index=False)
conn.close()